In [1]:
import requests, time, csv
import folium

In [2]:
def get_osm_data(csv_name, query_block):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json][timeout:25];
    area["ISO3166-1"="BE"][admin_level=2]->.belgium;
    (
    {query_block}
    );
    out center 150000;
    """
    
    while True:
        try:
            data = requests.post(overpass_url, data={"data": overpass_query}).json()
            features = [(e["tags"].get("name", "Unnamed"), e.get("lon", e.get("center", {}).get("lon")), 
                        e.get("lat", e.get("center", {}).get("lat")))
                        for e in data.get("elements", [])]

            with open(csv_name, "w", encoding="utf-8", newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["name", "lon", "lat"])
                writer.writerows(features)

            print(f"Saved {len(features)} locations to {csv_name}")
            break
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}, retrying in 10s...")
            time.sleep(10)
    

In [3]:
get_osm_data("belgium_parks.csv", """node["leisure"="park"](area.belgium);
  way["leisure"="park"](area.belgium);
  relation["leisure"="park"](area.belgium);""")

get_osm_data("belgium_bus_stops.csv", """node["highway"="bus_stop"](area.belgium);""")

get_osm_data("belgium_schools.csv", """node["amenity"="school"](area.belgium);
  way["amenity"="school"](area.belgium);
  relation["amenity"="school"](area.belgium);""")


Saved 6531 locations to belgium_parks.csv
Saved 60003 locations to belgium_bus_stops.csv
Saved 7166 locations to belgium_schools.csv


In [4]:
def csv_to_map(map_name, csv_file, location=[50.85, 4.35], zoom_start=8):
    map = folium.Map(location=location, zoom_start=zoom_start)
    with open(csv_file, encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader)
        for name, lon, lat in reader:
            folium.Marker(
                location=[float(lat), float(lon)],
                popup=name,
                icon=folium.Icon(color="green")
            ).add_to(map)
    map.save(map_name)
    print(f"Map saved as {map_name}")

In [5]:
csv_to_map("belgium_parks_map.html", "belgium_parks.csv")
csv_to_map("belgium_bus_stops_map.html", "belgium_bus_stops.csv")
csv_to_map("belgium_schools_map.html", "belgium_schools.csv")

Map saved as belgium_parks_map.html
Map saved as belgium_bus_stops_map.html
Map saved as belgium_schools_map.html


In [6]:
belgium_map_trains = folium.Map(location=[50.85, 4.35], zoom_start=8)

with open("belgium_train_stations.csv", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader) 
    for row in reader:
        name = row[2]
        lon = row[6]  
        lat = row[5] 
        folium.Marker(
            location=[float(lat), float(lon)],
            popup=name,
            icon=folium.Icon(color="green")
        ).add_to(belgium_map_trains)


belgium_map_trains.save("belgium_trains_map.html")
print(f"Map saved as belgium_trains_map.html")

Map saved as belgium_trains_map.html
